In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['ANGER', 'REquestion_to_question1_depression', 'actual_rewrite_angry', 'addiction', 'affToQuestoques4_depression', 'aff_distraction_addiction', 'aff_fs_addiction', 'aff_power_tool_addiction', 'aff_question2_angry', 'aff_question3_angry', 'affirmation_distraction_addiction', 'affirmation_negation_ques2_anxiety', 'affirmation_ques2_anxiety', 'affirmation_to_ques1_anxiety', 'affirmation_to_ques1_depression', 'affirmation_to_question2_depression', 'affirmation_to_question3_depression', 'afftoques4_depression', 'after_rewrite_angry', 'after_whos_island', 'almos_end_grief', 'anxiety', 'asking_name_grief', 'better_feel_angry', 'challenge_thought', 'challenge_thought

In [6]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\samit\MACHINE LEARNING\chatbot BE project\New\model.tflearn


In [7]:
yeah_dict = {'problem_got': ('response_to_condolence', 'How would you categorize this issue 1.Angry, 2.Anxiety, 3.Depression'), 'depressed':'response_to_condolence_depression', 'write_thought':'affirmation_to_ques1_depression', 'catastrophe':'affirmation_to_question2_depression', 'write_thought_anxiety':'affirmation_to_ques1_anxiety', 'future_assuming':'affirmation_ques2_anxiety', 'distortion':'affirmation_to_question3_depression', 'emotionalreasoning':'afftoques4_depression', 'write_thought_anxiety':'affirmation_to_ques1_anxiety', 'future_assuming':'affirmation_ques2_anxiety'}
no_dict = {'write_thought':'negation_to_ques1_depression', 'catastrophe':'negation_to_question2_depression','distortion':'negation_to_question4_depression', 'rewritethoughts':'negationtoques4_depression', 'write_thought_anxiety':'negation_to_ques1_anxiety', 'future_assuming':'affirmation_negation_ques2_anxiety', 'emotionalreasoning':'negationtoques4_depression'}
okay_dict = {'depressed':'response_to_condolence_depression','problem_got':'response_to_condolence','questoques1explained':'REquestion_to_question1_depression', 'questoques1recognized':'question2_depression', 'ques2explained':'requestion_to_question2_depression','makingUnderstandques2':'question3_depression','ques3explained':'got_it_depression', 'everythingExplained':'repquestion_to_question3_depression', 'gotchaques3':'question4_depression', 'ques4explained':'affToQuestoques4_depression', 'questoques1explained_anxiety':'reques_to_ques1_anxiety', 'questoques1confirmed_anxiety':'question2_anxiety', 'future_assuming_explanation':'reques_to_ques2_anxiety', 'affirmation_anxiety':'okay_ques2_anxiety'}

In [25]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=True):
    results = classify(sentence)
    for i in intents['intents']:
                # find a tag matching the first result
                if sentence in ['yes','yeah']:
                    print('in yeah loop')
                    tag = yeah_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
                        print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return(print('first yeah  ',resp))
                elif sentence in ['no','nope', 'not at all']:
                    print('in no loop')
                    tag = no_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
                        print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return(print('first  ',resp))
                elif sentence in ['okay','cool', 'gotcha', 'it is', 'got it', 'alright', 'it does']:
                    print('in okay loop')
                    tag = okay_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
                        print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return(print('first  ',resp))
    # if we have a classification then find the matching intent tag
    if results:
        print(results)
        # loop as long as there are matches to process
        for i in results:
            print(i)
            for j in intents['intents']:
                
                if i[0]==j['tag']:
                    try:
                        print('past context  ', context[userID])
                    except:
                        pass
                    if not 'context_filter' in j or \
                        (userID in context and 'context_filter' in j and j['context_filter'] == context[userID]):
                        print('in second if')
                        if show_details: print ('in first if tag:', j['tag'])
                        resp = random.choice(j['responses'])
                        if 'context_set' in j: 
                            context[userID] = j['context_set']
                            print('new context  ', context[userID])
                        return(resp)
                

        

else:
                    print('in else loop')
                    if 'context_filter' in j and j['context_filter'] == context[userID]:
                        resp = random.choice(j['responses'])
                        print('tag: ', j['tag'])
                        if 'context_set' in j: 
                            context[userID] = j['context_set']
                            print('new context  ', context[userID])
                        return(resp)

while results:
            for i in intents['intents']:
               
                    if i['tag'] == results[0][0]:
                        print('past context ',context, ' tag:', i['tag'])

                        if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                            if show_details: print ('in first if tag:', i['tag'])
                            resp = random.choice(i['responses'])

                        if 'context_filter' in i:
                            if show_details: print('context_filter in second if : ', i['context_filter'])
                            resp = random.choice(i['responses'])

                            # set context for this intent if necessary
                        if 'context_set' in i:
                            if show_details: print ('context in third if : ', i['context_set'])
                            context[userID] = i['context_set']
                            print('new context ', context)

                        if 'context_filter' in i:
                            if show_details: print('context_filter in fourth if : ', i['context_filter'])


                        return print('second  ' ,resp)


In [9]:
response('hi')

[('greeting', 0.9993413)]
('greeting', 0.9993413)
in second if
in first if tag: greeting


'Hello, How can I help you?'

In [10]:
response('help me feel better')

[('need_help', 0.9998609)]
('need_help', 0.9998609)
in second if
in first if tag: need_help
new context   problem_told


'Okay tell me your problem'

In [11]:
response('im having problem with')

[('stating_problem', 0.99998903)]
('stating_problem', 0.99998903)
past context   problem_told
in second if
in first if tag: stating_problem
new context   problem_got


'that sounds tough'

In [12]:
response('it is')

in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
new context   problem_got
first   Can you tell me how does it makes you feel? 1.Angry, 2.Anxiety, 3.Depression


In [13]:
response('depression')

[('depression', 1.0)]
('depression', 1.0)
past context   problem_got
in second if
in first if tag: depression
new context   depressed


'Oh gosh! That must be really tough'

In [14]:
response('it is')

in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
in okay loop
new context   depressed_1
first   You just wanted to share or do you want my help?


In [15]:
response('work on it')

In [16]:
response('challenge my thought')

[('challenge_thought', 0.59379566), ('challenge_thought_anxiety', 0.4012561)]
('challenge_thought', 0.59379566)
past context   depressed_1
('challenge_thought_anxiety', 0.4012561)
past context   depressed_1


In [17]:
response('i have this problem')

[('actual_rewrite_angry', 0.28005156), ('question1_depression', 0.2579449)]
('actual_rewrite_angry', 0.28005156)
past context   depressed_1
('question1_depression', 0.2579449)
past context   depressed_1


In [18]:
response('not at all')

in no loop


KeyError: 'depressed_1'

In [ ]:
response('catastrophe')

In [ ]:
response('it is')

In [ ]:
response('yes')

In [ ]:
response('got it')

In [ ]:
response('no')

In [ ]:
response('no')

In [26]:
response('hi')

[('greeting', 0.9993413)]
('greeting', 0.9993413)
in second if
in first if tag: greeting


"What's up what can I do for you?"

In [27]:
response('anxiety')

[('anxiety', 0.99994457)]
('anxiety', 0.99994457)
in second if
in first if tag: anxiety
new context   anxiety


'want my help or just wanted to share'

In [28]:
response('work on it')

In [29]:
response('challenge my thought')

[('challenge_thought', 0.59379566), ('challenge_thought_anxiety', 0.4012561)]
('challenge_thought', 0.59379566)
past context   anxiety
('challenge_thought_anxiety', 0.4012561)
past context   anxiety


In [30]:
response('i have this problem')

[('actual_rewrite_angry', 0.28005156), ('question1_depression', 0.2579449)]
('actual_rewrite_angry', 0.28005156)
past context   anxiety
('question1_depression', 0.2579449)
past context   anxiety


In [31]:
response('what is that?')

[('question_to_question3_depression', 0.27333075)]
('question_to_question3_depression', 0.27333075)
past context   anxiety


In [32]:
response('got it')

in okay loop


KeyError: 'anxiety'

In [ ]:
response('yes')

In [ ]:
response('okay')

In [ ]:
response('no')

In [ ]:
response('no')